In [ ]:
# !gcloud beta deployment-manager deployments create my-first-deployment --config vm.yaml --create-policy CREATE

In [14]:
dt.now().strftime('%m%d')

'0105'

In [3]:
from datetime import datetime as dt

In [ ]:
import pandas as pd
# import tensorflow.keras as keras
import os
from tqdm.notebook import tqdm as pbar
from plotly import express as px
from utils import plot, dollars_to_ticks
import numpy as np

In [ ]:
import h2o
from h2o.automl import H2OAutoML

In [ ]:
# annoying ssl warning (its insecure!)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
!gsutil cp ./analysis_sets/2019_Oct_trimmed.csv gs://temp_h2o/test.csv

In [ ]:
df = pd.read_csv('analysis_sets/2019_Oct_trimmed.csv')
df.shape

In [ ]:
df.head()

### Feature Eng

Working up:

* trailing features (shift)
* target for classification (expanding)
* trailing features (rolling)
* convert to ticks
* feature normalisation (pratical time series)
* vol norms by risk
* conv net + lstm

Notes:

* The event location is a potential feature as different meets may behave similairly, likewise the race number and the day of the meet - you will need more data for this though.

In [ ]:
df.fillna(0, inplace=True)

keep_cols = ['timestamp', 'market_id', 'sid', 'batl', 'batb', 'batl_v', 'batb_v',
       'batl_v_1', 'batb_v_1', 'batl_v_2', 'batb_v_2', 'batl_v_3', 'batb_v_3',
       'batl_v_4', 'batb_v_4', 'batl_v_5', 'batb_v_5', 'tot_vol', 'mean_trd']

df = df[keep_cols]

df['mid'] = (df.batb + df.batl) / 2

price_cols = ['batl', 'batb', 'mean_trd', 'mid']
for c in price_cols:
    df[f'{c}_ticks'] = df[c].apply(dollars_to_ticks)

df['spread'] = df.batl_ticks - df.batb_ticks

df['diff'] = df.groupby(['market_id', 'sid']).mid_ticks.diff().cumsum()

# temp = df[df.spread==1]
df['diff_ticks'] = df.groupby(['market_id', 'sid']).mid_ticks.diff()
df['diff_ticks'] = df.groupby(['market_id', 'sid']).diff_ticks.cumsum()

df['trade_cnt'] = 1
df['trade_cnt'] = df.groupby(['market_id', 'sid']).trade_cnt.cumsum()

vals = {}
idxs = {}
for sid in df.sid.unique():
    vals[sid] = df[df.sid==sid].mid_ticks.values
    idxs[sid] = df[df.sid==sid].index.values

def find_pos(row):
    val = row.mid_ticks + 5
    idx = np.argwhere(idxs[row.sid] == row.inc_idx).flatten()[0]
    val = np.argmax(vals[row.sid][idx:] >= val)
    return val

def find_neg(row):
    val = row.mid_ticks - 5
    idx = np.argwhere(idxs[row.sid] == row.inc_idx).flatten()[0]
    val = np.argmax(vals[row.sid][idx:] <= val)
    return val

df['inc_idx'] = df.index

df['pos_idx'] = df.apply(find_pos, axis=1)

df['neg_idx'] = df.apply(find_neg, axis=1)

df['label'] = df.pos_idx < df.neg_idx

for x in range(1, 50):
    df[f'ticks_{x}'] = df.mid_ticks - df.groupby(['market_id', 'sid']).mid_ticks.shift(x)

df['ticks_from_mean'] = df.mid_ticks - df.mean_trd_ticks

In [ ]:
feature_cols = ['batl_v', 'batb_v',
       'batl_v_1', 'batb_v_1', 'batl_v_2', 'batb_v_2', 'batl_v_3', 'batb_v_3',
       'batl_v_4', 'batb_v_4', 'batl_v_5', 'batb_v_5', 'ticks_from_mean',
       'spread', 'ticks_1', 'ticks_2', 'ticks_3', 'ticks_4',
       'ticks_5', 'ticks_6', 'ticks_7', 'ticks_8', 'ticks_9', 'ticks_10',
       'ticks_11', 'ticks_12', 'ticks_13', 'ticks_14', 'ticks_15', 'ticks_16',
       'ticks_17', 'ticks_18', 'ticks_19', 'ticks_20', 'ticks_21', 'ticks_22',
       'ticks_23', 'ticks_24', 'ticks_25', 'ticks_26', 'ticks_27', 'ticks_28',
       'ticks_29', 'ticks_30', 'ticks_31', 'ticks_32', 'ticks_33', 'ticks_34',
       'ticks_35', 'ticks_36', 'ticks_37', 'ticks_38', 'ticks_39', 'ticks_40',
       'ticks_41', 'ticks_42', 'ticks_43', 'ticks_44', 'ticks_45', 'ticks_46',
       'ticks_47', 'ticks_48', 'ticks_49']

y_name = 'label'

In [ ]:
# df = pd.read_csv('analysis_sets/train_sets/oct_2019.csv')

In [ ]:
pass_df = df[feature_cols + [y_name]].dropna()
pass_df.shape

In [ ]:
h2o.connect(url='https://34.67.110.23:443', auth=('h2oai', '6978787547046774965'), verify_ssl_certificates=False)

In [ ]:
pass_df.to_csv('./test.csv')
!gsutil cp ./test.csv gs://temp_h2o/test.csv

In [ ]:
hf = h2o.import_file('gs://temp_h2o/test.csv')

In [ ]:
# h2o.init()

In [ ]:
# hf = h2o.H2OFrame(pass_df)

In [ ]:
hf = hf.drop('C1', axis=1)

In [ ]:
hf[y_name] = hf[y_name].asfactor()

In [ ]:
# hf.as_data_frame().to_csv('analysis_sets/train_sets/oct_2019.csv')

In [ ]:
# del df
# del pass_df

In [ ]:
# model = h2o.import_mojo('GBM_3_AutoML_20191228_003853.zip')

In [ ]:
# perf = model.model_performance(hf)
# perf

In [ ]:
# help(H2OAutoML)

In [ ]:
hf.shape

In [ ]:
aml = H2OAutoML(max_models=20, seed=42)
# runtime = 60*50
# H2OAutoML(max_runtime_secs=runtime, seed=42)
aml.train(x=feature_cols, y=y_name, training_frame=hf)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
# h2o.get_model('AutoML_Leaderboard_test')

In [ ]:
h2o.ls()

In [ ]:
aml.leader.varimp_plot()

In [ ]:
new_model_path = h2o.save_model(h2o_glm, "hdfs://" + hdfs_name_node + "/" + hdfs_model_path)

In [ ]:
h2o.shutdown()

In [ ]:
aml.leader.download_mojo()

In [ ]:
preds = aml.leader.predict(hf)

In [ ]:
df_ext = preds.as_data_frame()

In [ ]:
df_ext.head()

In [ ]:
for c in df_ext.columns:
    pass_df[c] = df_ext[c]

In [ ]:
(pass_df.label == pass_df.predict).sum() / len(pass_df)

In [ ]:
df_ext.shape, pass_df.shape

In [ ]:
h2o.shutdown()

In [ ]:
!gcloud deployment-manager deployments delete h2oai-h2o3-cluster-launcher-1 --delete-policy=DELETE